In [1]:
import os
import cv2
import numpy as np
import tensorflow.keras as keras
import tensorflow as tf
from tensorflow.keras.layers import Lambda,Dense,ReLU,MaxPooling2D,Flatten,Add
from tensorflow.keras.layers import Dense,Input,MaxPool2D,Conv2D,Dropout,Concatenate
import matplotlib.pyplot as plt
from utils import *
from tensorflow.keras.datasets import mnist
from tensorflow.keras.models import Model

(x_train, y_train), (x_test, y_test) = keras.datasets.mnist.load_data()

In [2]:
num_classes = 10
input_shape = (28, 28, 1)
(x_train, y_train), (x_test, y_test) = keras.datasets.mnist.load_data()

x_train = x_train.astype("float32") / 255
x_test = x_test.astype("float32") / 255

x_train = np.expand_dims(x_train, -1)
x_test = np.expand_dims(x_test, -1)
print("x_train shape:", x_train.shape)
print(x_train.shape[0], "train samples")
print(x_test.shape[0], "test samples")

# convert class vectors to binary class matrices
y_train = keras.utils.to_categorical(y_train, num_classes)
y_test = keras.utils.to_categorical(y_test, num_classes)

x_train shape: (60000, 28, 28, 1)
60000 train samples
10000 test samples


In [3]:
inputs = Input(shape=(28, 28, 1))

output_1 = Involution(channel=3, group_number=1, kernel_size=3, stride=1,
                      reduction_ratio=2, name="involution_1")(inputs)
output_1 = ReLU()(output_1)
output_1 = Concatenate()([output_1, inputs])

output_1 = Involution(channel=3, group_number=1, kernel_size=3, stride=1,
                      reduction_ratio=2, name="involution_2")(output_1)
output_1 = ReLU()(output_1)

output = Concatenate()([output_1, inputs])

output = Flatten()(output)
output = Dropout(0.5)(output)
output = Dense(num_classes, activation="softmax")(output)
model = Model(inputs=inputs, outputs=output)

model.summary()
# keras.utils.plot_model(model, to_file='model.png')


Model: "functional_1"
__________________________________________________________________________________________________
Layer (type)                    Output Shape         Param #     Connected to                     
input_1 (InputLayer)            [(None, 28, 28, 1)]  0                                            
__________________________________________________________________________________________________
involution_1 (Involution)       (None, 28, 28, 1)    24          input_1[0][0]                    
__________________________________________________________________________________________________
re_lu (ReLU)                    (None, 28, 28, 1)    0           involution_1[0][0]               
__________________________________________________________________________________________________
concatenate (Concatenate)       (None, 28, 28, 2)    0           re_lu[0][0]                      
                                                                 input_1[0][0]         

In [4]:
batch_size = 128
epochs = 20
model.compile(loss="categorical_crossentropy", optimizer="adam", metrics=["accuracy"])
model.fit(x_train, y_train, batch_size=batch_size, epochs=epochs, validation_split=0.1)

Epoch 1/20
422/422 [==============================] - 12s 27ms/step - loss: 0.5614 - accuracy: 0.8342 - val_loss: 0.2436 - val_accuracy: 0.9340
Epoch 2/20
422/422 [==============================] - 13s 30ms/step - loss: 0.3349 - accuracy: 0.9018 - val_loss: 0.1966 - val_accuracy: 0.9448
Epoch 3/20
422/422 [==============================] - 13s 31ms/step - loss: 0.2772 - accuracy: 0.9191 - val_loss: 0.1517 - val_accuracy: 0.9590
Epoch 4/20
422/422 [==============================] - 13s 31ms/step - loss: 0.2368 - accuracy: 0.9302 - val_loss: 0.1343 - val_accuracy: 0.9643
Epoch 5/20
422/422 [==============================] - 13s 31ms/step - loss: 0.2115 - accuracy: 0.9366 - val_loss: 0.1264 - val_accuracy: 0.9630
Epoch 6/20
422/422 [==============================] - 13s 32ms/step - loss: 0.1995 - accuracy: 0.9399 - val_loss: 0.1239 - val_accuracy: 0.9652
Epoch 7/20
422/422 [==============================] - 13s 32ms/step - loss: 0.1901 - accuracy: 0.9421 - val_loss: 0.1127 - val_accuracy:

In [5]:
score = model.evaluate(x_test, y_test, verbose=0)
score

[0.11619466543197632, 0.9670000076293945]